In [1]:
!pip3 install langchain
!pip3 install accelerate
!pip3 install bitsandbytes
!pip install gpt4all 
!pip install transformers

!pip install numexpr
!pip install google-search-results

!pip install langchain --upgrade


In [2]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate


from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


import pandas as pd


/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading the data to pandas dataframe

In [3]:
data_path = "archive/test.csv"
df = pd.read_csv(data_path)
df

,url,question,answers,Unnamed: 3,result_with_tools,result_without_tools,errors
0,http://www.freebase.com/view/en/jamaica,what does jamaican people speak?,['Jamaican Creole English Language' 'Jamaican ...,NaN,1.0,NaN,NaN
1,http://www.freebase.com/view/en/james_k_polk,what did james k polk do before he was president?,['Lawyer'],NaN,NaN,NaN,NaN
2,http://www.freebase.com/view/en/oregon_ducks,what is the oregon ducks 2012 football schedule?,['University of Oregon'],NaN,NaN,NaN,NaN
3,http://www.freebase.com/view/en/ken_barlow,who plays ken barlow in coronation street?,['Tony Warren'],NaN,NaN,NaN,NaN
4,http://www.freebase.com/view/en/chiune_sugihara,what happened after mr. sugihara died?,['Yaotsu'],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2027,http://www.freebase.com/view/en/david_beckham,what team did david beckham play for before la...,['Preston North End F.C.'],NaN,NaN,NaN,NaN
2028,http://www.freebase.com/view/en/france,who is the current leader of france 2010?,['Nicolas Sarkozy'],NaN,NaN,NaN,NaN
2029,http://www.freebase.com/view/en/knossos,where was the palace of knossos located?,['Crete' 'Greece'],NaN,NaN,NaN,NaN
2030,http://www.freebase.com/view/en/roswell_ufo_in...,where is roswell area 51?,['Roswell'],NaN,NaN,NaN,NaN


Adding local path for general llm and for evaluation

In [4]:
local_path = ("/home/naheed/Desktop/gpt4all/model/mistral-7b-openorca.Q4_0.gguf") #gpt4all
# local_path = ("/home/naheed/Desktop/gpt4all/ggml-model-q4_0.gguf") #TinyLlama/TinyLlama-1.1B-Chat-v0.6
# local_path = ("G:\gpt4\model\ggml-model-q4_0.gguf") #TinyLlama/TinyLlama-1.1B-Chat-v0.6
eval_local_path = ("/home/naheed/Desktop/gpt4all/model/mistral-7b-openorca.Q4_0.gguf") #gpt4all
# eval_local_path = ("G:\gpt4\mistral-7b-openorca.Q4_0.gguf") #TinyLlama/TinyLlama-1.1B-Chat-v0.6
# eval_local_path = ("G:\gpt4\model\ggml-model-q4_0.gguf") #TinyLlama/TinyLlama-1.1B-Chat-v0.6

In [5]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
# llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True , streaming=True)
eval_llm = GPT4All(model=eval_local_path, callbacks=callbacks, verbose=True , streaming=True)
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)
# eval_llm = GPT4All(model=eval_local_path,device='gpu', callbacks=callbacks, verbose=False)
# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

instantiating model for sentiment analysis

In [6]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
custom_promt = """
Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current news
Calculator: useful for when you need to answer questions about math

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

"""

In [8]:
import os
import pprint
%env SERPAPI_API_KEY = "e1f8cd9de27d3b78f9ab3428167883002e76c33b"
os.environ["SERPER_API_KEY"] = "e1f8cd9de27d3b78f9ab3428167883002e76c33b"

from langchain.utilities import GoogleSerperAPIWrapper
search = GoogleSerperAPIWrapper()


env: SERPAPI_API_KEY="e1f8cd9de27d3b78f9ab3428167883002e76c33b"


In [9]:


from langchain.tools import Tool
from langchain.chains import LLMMathChain

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from langchain.agents import create_react_agent, AgentExecutor

from langchain.chains import LLMChain



llm_math_chain = LLMMathChain(llm=llm, verbose=True)

google_search = Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events",
        # description="useful for when you need to answer questions about current news",
        # coroutine= ... <- you can specify an async method if desired as well
    )

calculator =  Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        # coroutine= ... <- you can specify an async method if desired as well
    )

tools = [ google_search , calculator ]


# # conversational agent memory
# memory = ConversationBufferWindowMemory(
#     memory_key='chat_history',
#     k=3,
#     return_messages=True
# )


/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [10]:
# Set up the base template
template_asking_with_tools = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! 

New question: {question}
{agent_scratchpad}"""


template_for_comapring_answers = """Answer the following questions as best you can.


Use the following format:

Question: the input question you must answer
Final Answer: the final answer to the original input question


Promt1: "{llm_generated}" 
Promt2: "{actual_answer}"

Begin! 

Question: "{question}" Does Promt1 answer matches with Promt2 . Answer it in only Yes or No.
"""



prompt1_with_tools = PromptTemplate(
    template=template_asking_with_tools,
    tools=tools,
    input_variables=["question"]
)

prompt2_for_comparison = PromptTemplate(
    template=template_for_comapring_answers,
    input_variables=["question", "llm_generated" , "actual_answer"]
)



In [11]:
# print(df)
df

,url,question,answers,Unnamed: 3,result_with_tools,result_without_tools,errors
0,http://www.freebase.com/view/en/jamaica,what does jamaican people speak?,['Jamaican Creole English Language' 'Jamaican ...,NaN,1.0,NaN,NaN
1,http://www.freebase.com/view/en/james_k_polk,what did james k polk do before he was president?,['Lawyer'],NaN,NaN,NaN,NaN
2,http://www.freebase.com/view/en/oregon_ducks,what is the oregon ducks 2012 football schedule?,['University of Oregon'],NaN,NaN,NaN,NaN
3,http://www.freebase.com/view/en/ken_barlow,who plays ken barlow in coronation street?,['Tony Warren'],NaN,NaN,NaN,NaN
4,http://www.freebase.com/view/en/chiune_sugihara,what happened after mr. sugihara died?,['Yaotsu'],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2027,http://www.freebase.com/view/en/david_beckham,what team did david beckham play for before la...,['Preston North End F.C.'],NaN,NaN,NaN,NaN
2028,http://www.freebase.com/view/en/france,who is the current leader of france 2010?,['Nicolas Sarkozy'],NaN,NaN,NaN,NaN
2029,http://www.freebase.com/view/en/knossos,where was the palace of knossos located?,['Crete' 'Greece'],NaN,NaN,NaN,NaN
2030,http://www.freebase.com/view/en/roswell_ufo_in...,where is roswell area 51?,['Roswell'],NaN,NaN,NaN,NaN


In [12]:
# for some warning
# pd.options.mode.chained_assignment = None  # default='warn'

# df
# creating columns
df['result_with_tools']=""
df['result_without_tools']=""
df['errors']=""

start = 0 
finish = 200
df
# print(df.to_string)

for i in range(start,finish):
    

    question = df['question'][i]
    actual_answer = df['answers'][i]
    print(str(i)  +" Question    "+ question)
    print(str(i)  +" Actual Ans: "+ actual_answer)




    # llm_chain1 = LLMChain(llm=llm, prompt=prompt1_with_tools)

    tool_names = [tool.name for tool in tools]
    agent = create_react_agent(

        tools=tools ,
        prompt=prompt1_with_tools,
        llm = llm,
    )

    try:
        agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True,handle_parsing_errors=True,max_iterations=3)
        llm_generated = agent_executor.invoke({"question": question})
       
    except Exception as e:
        print(e)
        df.loc[i,'errors']=1

    # print("==========================================================")
    
    
   

    try:
        llm_chain2 = LLMChain(llm=eval_llm, prompt=prompt2_for_comparison)
        final_result = llm_chain2.invoke({"question": question , "llm_generated":llm_generated['output'] , "actual_answer" :actual_answer})

    except Exception as e:
        print(e)
        df.loc[i,'errors']=1


    print(final_result)
    classifier = pipeline("sentiment-analysis" , model = model,tokenizer=tokenizer)
    print("\n-----------------------sentiment------------------------------------")
    label = classifier([str(final_result['text'])])
    print(label[0]["label"])
    print(label[0])


    # label[0]["label"]


    if(label[0]["label"] == "POSITIVE"):
        df.loc[i,'result_with_tools']= 1
        print("TRUE")
    else:
        df.loc[i,'result_with_tools']= 0
        print("FALSE")

    df.to_csv(data_path ,index = False)
    print("Data saved")



print("Process finished")
df = pd.read_csv(data_path , index = False)
df


0 Question    what does jamaican people speak?
0 Actual Ans: ['Jamaican Creole English Language' 'Jamaican English']


> Entering new AgentExecutor chain...

Thought: I need to find out which language Jamaicans speak.
Action: Search
Action Input: "Jamaican language"
Observation: The official language of Jamaica is English, but many Jamaicans also speak Jamaican Patois or simply Patois.

Thought: Now that I know the languages spoken in Jamaica, I can provide an answer to the question.
Final Answer: Jamaican people speak English and some also speak Jamaican Patois.
Thought: I need to find out which language Jamaicans speak.
Action: Search
Action Input: "Jamaican language"Our Language Jamaica's official language is English, but we also speak Jamaican or Patois (or Patwa) - a colorful, descriptive and emphatic creole dialect that has been shaped by our African, Spanish, French, Portuguese, and English colonial heritage. I now know the final answer.
Final Answer: Jamaicans speak English and

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7f9b2f2ba7a0>
Traceback (most recent call last):
  File "/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 479, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 
